# RAG - retrieval augmented generation

<!--<badge>--><a href="https://colab.research.google.com/github/kuennethgroup/colab_tutorials/blob/main/lecture11/rag_for_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

- llama-index for building the RAG pipeline
- we want to ask question related to wikipedia articles on polymers


In [1]:
import os

# Restrict GPU usage to 2 and 3
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

## Download polymer related information from wikipedia

We use the wikipediaapi that create urls for pages

In [2]:
import wikipediaapi

polymer_wiki = [
    "Acrylonitrile butadiene styrene",
    "Cross-linked polyethylene",
    "Ethylene vinyl acetate",
    "Poly(methyl methacrylate)",
    "Poly(ethyl methacrylate)",
    "Polyacrylic acid",
    "Polyamide",
    "Polybutylene",
    "Polybutylene terephthalate",
    "Polycarbonate",
    "Polyetheretherketone",
    "Polyester",
    "Polyethylene",
    "Polyethylene terephthalate",
    "Polyimide",
    "Polylactic acid",
    "Polyoxymethylene",
    "Polyphenyl ether",
    "Poly(p-phenylene oxide)",
    "Polypropylene",
    "Polystyrene",
    "Polysulfone",
    "Polytetrafluoroethylene",
    "Polyurethane",
    "Polyvinyl chloride",
    "Polyvinylidene chloride",
    "Styrene maleic anhydride",
    "Styrene-acrylonitrile",
    "Tritan copolyester",
    "Comonomer",
    "Copolymer",
    "Acrylonitrile butadiene styrene",
    "Alginic acid",
    "Arabinoxylan",
    "Azoximer bromide",
    "Bovhyaluronidase azoximer",
    "Citroën Méhari",
    "Cyclic olefin copolymer",
    "Dispersity",
    "Dynel",
    "ECTFE",
    "ETFE",
    "Ethylene copolymer bitumen",
    "Ethylene vinyl alcohol",
    "Ethylene-vinyl acetate",
    "Fluorinated ethylene propylene",
    "Gradient copolymer",
    "Ionomer",
    "Kraton (polymer)",
    "Merrifield resin",
    "Methacrylate copolymer",
    "Nitrile rubber",
    "P123",
    "Paraloid B-72",
    "Polybutadiene acrylonitrile",
    "PEDOT:PSS",
    "PHBV",
    "PLGA",
    "Polilactofate",
    "Polyacrylonitrile",
    "Polydiethylstilbestrol phosphate",
    "Polyestradiol phosphate",
    "Polyestriol phosphate",
    "Polyether block amide",
    "Polytestosterone phloretin phosphate",
    "Polyvinyl chloride acetate",
    "Solvent vapour annealing",
    "Spandex",
    "Styrene maleic anhydride",
    "Styrene-acrylonitrile resin",
    "Styrene-butadiene",
]

polymer_wiki = [w.replace(" ", "_") for w in polymer_wiki]

wiki_wiki = wikipediaapi.Wikipedia("polymers", "en")
urls = []
for page in polymer_wiki:
    page_py = wiki_wiki.page(page)
    urls.append(page_py.fullurl)

# Remove duplicates
urls = list(set(urls))

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

In [6]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

/home/chris/courses/st24_MLinMS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chris/courses/st24_MLinMS/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


2024-07-17 12:18:10.720492: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 12:18:10.720578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 12:18:10.723044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 12:18:10.731284: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 12:18:11.467106: W tensorflow/compiler/tf2

INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [13]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_K_M.gguf"
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

ModuleNotFoundError: No module named 'llama_index.llms.llama_cpp'

In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

from llama_index import (
    VectorStoreIndex,
    load_index_from_storage,
    StorageContext,
)


# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleWebPageReader(html_to_text=True).load_data(urls)
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context, service_context=service_context)

query_engine = index.as_query_engine()

In [ ]:
# set Logging to DEBUG for more detailed outputs
response = query_engine.query("What are the Tgs of PE?")
display(Markdown(f"{response}"))


llama_print_timings:        load time =   10120.94 ms
llama_print_timings:      sample time =      37.37 ms /   204 runs   (    0.18 ms per token,  5458.34 tokens per second)
llama_print_timings: prompt eval time =   20409.76 ms /   956 tokens (   21.35 ms per token,    46.84 tokens per second)
llama_print_timings:        eval time =   87276.16 ms /   203 runs   (  429.93 ms per token,     2.33 tokens per second)
llama_print_timings:       total time =  108172.56 ms /  1159 tokens


  Based on the provided context information, the Tg (glass transition temperature) of PEBA (Polyether block amide) is not explicitly mentioned. However, it can be inferred that the Tg of PEBA is likely to be relatively high due to its high molecular weight and crystalline structure.

The Tg of a polymer is an important property that determines its thermal stability and mechanical behavior. In general, high Tg materials tend to have better thermal stability and resistance to degradation, but may exhibit more brittle behavior at lower temperatures.

Since PEBA is used in various applications such as sports equipment, medical products, and electronic goods, it is likely that the Tg of PEBA is designed to be relatively high to ensure its thermal stability and mechanical properties under different operating conditions. However, without specific information on the Tg of PEBA, it is not possible to provide a more precise answer.

In [ ]:
# set Logging to DEBUG for more detailed outputs
response = query_engine.query(
    "Return all the properties mentioned for polyethylene as table "
)
display(Markdown(f"{response}"))

Llama.generate: prefix-match hit



  Sure! Based on the provided context information, here are the properties of polyethylene mentioned in the text:

| Property | Description |
| --- | --- |
| Density | The density of polyethylene depends significantly on variables such as the extent and type of branching, the crystal structure, and the molecular weight. |
| Branching | Polyethylene can have different types of branching, which affect its mechanical properties. |
| Molecular weight | The molecular weight of polyethylene is measured in millions of atomic mass units (amu), typically between 3.5 and 7.5 million amu. |
| Crystal structure | The crystal structure of polyethylene affects its mechanical properties. |
| Ultra-high-molecular-weight | Polyethylene with a molecular weight numbering in the millions, usually between 3.5 and 7.5 million amu, is called ultra-high-molecular-weight polyethylene (UHMWPE). |
| High-density | Polyethylene with a higher density than UHMWPE is called high-

llama_print_timings:        load time =   10120.94 ms
llama_print_timings:      sample time =      44.17 ms /   256 runs   (    0.17 ms per token,  5795.26 tokens per second)
llama_print_timings: prompt eval time =   19625.30 ms /   991 tokens (   19.80 ms per token,    50.50 tokens per second)
llama_print_timings:        eval time =  109502.29 ms /   255 runs   (  429.42 ms per token,     2.33 tokens per second)
llama_print_timings:       total time =  129760.05 ms /  1246 tokens


In [ ]:
# set Logging to DEBUG for more detailed outputs
response = query_engine.query("What are the Tgs of PE? Return as Markdown table")
display(Markdown(f"{response}"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10120.94 ms
llama_print_timings:      sample time =      40.13 ms /   234 runs   (    0.17 ms per token,  5831.34 tokens per second)
llama_print_timings: prompt eval time =   19015.47 ms /   921 tokens (   20.65 ms per token,    48.43 tokens per second)
llama_print_timings:        eval time =   99051.76 ms /   233 runs   (  425.11 ms per token,     2.35 tokens per second)
llama_print_timings:       total time =  118641.22 ms /  1154 tokens


  Sure! Based on the provided context information, I can answer your query.

The Tg (glass transition temperature) of PE (polyethylene) is not a fixed value and can vary depending on the specific type of PE and its manufacturing process. However, here is a general breakdown of the typical Tg ranges for different types of PE:

| Type of PE | Tg Range (°C) |
| --- | --- |
| LLDPE (Linear Low-Density Polyethylene) | 40-60 |
| HDPE (High-Density Polyethylene) | 80-120 |
| PP (Polypropylene) | 120-160 |
| PVC (Polyvinyl Chloride) | 180-240 |

Please note that these are general ranges and the actual Tg of a specific material can vary depending on factors such as molecular weight, crystallinity, and additives.